# Creating feature matrix

In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
import csv
import re
from shapely.geometry import box, Point
import glob 
import os

## Preparing Datasets

### Shapefile

In [ ]:
# Read borough shapefile
nybb = gpd.read_file("nyc_shapefile/nybb.shp")

print(nybb.crs)
# this should be EPSG:2263  (NAD 1983 StatePlane New York Long Island, feet)

# Dissolve boroughs into a single NYC polygon
nyc = nybb.dissolve()          # one row, geometry is union of all boroughs
nyc = nyc.reset_index(drop=True)

# Get bounding box of NYC in feet (since CRS is in feet)
minx, miny, maxx, maxy = nyc.total_bounds

# Cell size: 2 km in feet (using US survey foot)
feet_per_meter = 1 / 0.3048006096012192
cell_size_ft = 2000 * feet_per_meter   # about 6561.67 feet

print(f"Cell size in feet: {cell_size_ft:.2f}")

# Build grid of square polygons over the bounding box
xs = np.arange(minx, maxx + cell_size_ft, cell_size_ft)
ys = np.arange(miny, maxy + cell_size_ft, cell_size_ft)

cells = []
for x in xs:
    for y in ys:
        cell = box(x, y, x + cell_size_ft, y + cell_size_ft)
        cells.append(cell)

grid = gpd.GeoDataFrame({"geometry": cells}, crs=nyc.crs)

print(f"Grid has {len(grid)} raw cells before clipping")

# Clip grid to NYC outline
grid_clipped = gpd.clip(grid, nyc)

# Remove empty geometries if any
grid_clipped = grid_clipped[~grid_clipped.is_empty].reset_index(drop=True)

# Add region_id
grid_clipped["region_id"] = grid_clipped.index

print(f"Grid has {len(grid_clipped)} cells after clipping")

# Inspect
print(grid_clipped.head())

# Save shapefile
grid_clipped.to_file("data/nyc_grid_2km.shp")


EPSG:2263
Cell size in feet: 6561.67
Grid has 625 raw cells before clipping
Grid has 285 cells after clipping
                                            geometry  region_id
0  POLYGON ((919736.776 126690.037, 926298.442 12...          0
1  POLYGON ((926298.442 126690.037, 931219.36 126...          1
2  POLYGON ((919736.776 126690.037, 919736.776 12...          2
3  POLYGON ((919736.776 133251.703, 919736.776 12...          3
4  POLYGON ((932860.109 133251.703, 939421.776 13...          4


### NYPD complaint data

In [11]:
complaint = pd.read_csv('old_data/NYPD_Complaint_Data.csv')

# turn empty strings into NaN
complaint = complaint.replace("", np.nan)

# if xcoord and ycoord are strings that sometimes contain spaces:
complaint["Lat_Lon"] = complaint["Lat_Lon"].astype(str).str.strip()
complaint["Date"] = complaint["CMPLNT_FR_DT"].astype(str).str.strip()
complaint = complaint.replace({"Lat_Lon": {"": np.nan},
                         "Date": {"": np.nan}})

# drop true missing values
complaint = complaint.dropna(subset=["Date", "Lat_Lon"])

# filter columns
complaint = complaint[['CMPLNT_FR_DT', 'Date', 'Lat_Lon']].copy()

# save
complaint.to_csv("data/Complaint.csv", index=False)

complaint.head()

,CMPLNT_FR_DT,Date,Lat_Lon
0,07/01/2012,07/01/2012,"(40.8441566000203, -73.9006054489734)"
1,07/01/2012,07/01/2012,"(40.6744956865259, -73.9305713255961)"
2,07/01/2012,07/01/2012,"(40.6984738177025, -73.917768981221)"
3,07/01/2012,07/01/2012,"(40.7565675846374, -73.8759315341335)"
4,07/01/2012,07/01/2012,"(40.7072398161698, -73.7927267255908)"


### Search and frisk dataset (SAS)

In [ ]:
sas12 = pd.read_csv("old_data/SAS_2012.csv")
sas13 = pd.read_csv("old_data/SAS_2013.csv")

def add_date_column(df):
    # to string
    s = df["datestop"].astype(str)
    # pad to 8 characters, e.g. "1012012" -> "01012012"
    s = s.str.zfill(8)
    # parse as MMDDYYYY
    df["date"] = pd.to_datetime(s, format="%m%d%Y")
    return df

sas12 = add_date_column(sas12)
sas13 = add_date_column(sas13)

# combine both years
sas = pd.concat([sas12, sas13], ignore_index=True)

# filter to 2012-07-01 through 2013-06-30
start = pd.Timestamp("2012-07-01")
end   = pd.Timestamp("2013-06-30")

sas_df = sas[(sas["date"] >= start) & (sas["date"] <= end)]

# turn empty strings into NaN
sas_df = sas_df.replace("", np.nan)

# if xcoord and ycoord are strings that sometimes contain spaces:
sas_df["xcoord"] = sas_df["xcoord"].astype(str).str.strip()
sas_df["ycoord"] = sas_df["ycoord"].astype(str).str.strip()
sas_df["Date"] = sas_df["date"].astype(str).str.strip()
sas_df = sas_df.replace({"xcoord": {"": np.nan},
                         "ycoord": {"": np.nan},
                         "date": {"": np.nan}})

# drop true missing values
sas_df = sas_df.dropna(subset=["Date", "xcoord", "ycoord"])

# filter columns
sas_df = sas_df[['Date', 'xcoord', 'ycoord']].copy()

# save
sas_df.to_csv("data/SAS.csv", index=False)

sas_df.head()


/var/folders/6f/lxkzpyyn0fvcj0qtkc53g_2w0000gn/T/ipykernel_34226/4071828253.py:1: DtypeWarning: Columns (10,73,103,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  sas12 = pd.read_csv("old_data/SAS_2012.csv")
/var/folders/6f/lxkzpyyn0fvcj0qtkc53g_2w0000gn/T/ipykernel_34226/4071828253.py:2: DtypeWarning: Columns (10,73,83) have mixed types. Specify dtype option on import or set low_memory=False.
  sas13 = pd.read_csv("old_data/SAS_2013.csv")


,Date,xcoord,ycoord
337410,2012-07-02,1019585,184371
337411,2012-07-03,987078,215157
337412,2012-07-05,1002416,231297
337414,2012-07-06,988511,164316
337415,2012-07-08,995824,230943


### 311 

In [ ]:
df_311 = pd.read_csv("old_data/311_Service_Requests.csv")

# Parse date directly, coercing bad ones to NaN
df_311["Date"] = pd.to_datetime(df_311["Created Date"], errors="coerce")

# Copy Location to Lat_Lon without astype(str) first
df_311["Lat_Lon"] = df_311["Location"]

# Treat blanks / whitespace / obvious junk as missing
df_311["Lat_Lon"] = df_311["Lat_Lon"].replace(
    [ "", " ", "  ", "nan", "NaN" ], np.nan
)
# also strip whitespace then turn pure-whitespace into NaN
df_311["Lat_Lon"] = df_311["Lat_Lon"].astype(str).str.strip()
df_311["Lat_Lon"] = df_311["Lat_Lon"].replace({"": np.nan, "nan": np.nan, "NaN": np.nan})

# Drop rows missing either Date or Lat_Lon
df_311 = df_311.dropna(subset=["Date", "Lat_Lon"])

# Keep only the needed columns
df_311 = df_311[["Unique Key", "Date", "Lat_Lon"]].copy()

df_311.to_csv("data/311.csv", index=False)

df_311.head()


,Unique Key,Date,Lat_Lon
0,25847928,2013-06-30 12:09:56,"(40.704084501215796, -73.90740226066829)"
1,25849947,2013-06-30 12:09:55,"(40.69569076684871, -73.73678889778218)"
3,25850028,2013-06-30 12:09:34,"(40.72260022794115, -73.86761631518405)"
4,25847685,2013-06-30 12:08:43,"(40.639048553176586, -74.00332571465987)"
5,25847702,2013-06-30 12:08:19,"(40.77814711286228, -73.9196478173007)"


### Weather

In [ ]:
# Read and normalize column names
weather = pd.read_csv("old_data/NYC_Central_Park_weather_1869-2022.csv")
weather.columns = weather.columns.str.strip().str.upper()

# Parse dates and sort
weather["DATE"] = pd.to_datetime(weather["DATE"])
weather = weather.sort_values("DATE")

# Keep only time window
start = pd.Timestamp("2012-07-01")
end   = pd.Timestamp("2013-06-30")

weather = weather[(weather["DATE"] >= start) & (weather["DATE"] <= end)].copy()

# Make sure to have a row for every day
all_days = pd.date_range(start, end, freq="D")
weather = weather.set_index("DATE").reindex(all_days)
weather.index.name = "DATE"

# Handle missing values
# treat missing precipitation and snow as 0
for col in ["PRCP", "SNOW", "SNWD"]:
    if col in weather.columns:
        weather[col] = weather[col].fillna(0.0)

# interpolate missing temps
for col in ["TMIN", "TMAX"]:
    if col in weather.columns:
        weather[col] = weather[col].interpolate(limit_direction="both")


weather_clean = weather[["PRCP", "SNOW", "TMIN", "TMAX"]].copy()

weather_clean.to_csv("data/weather.csv")

weather_clean.head()


,PRCP,SNOW,TMIN,TMAX
DATE,,,,
2012-07-01,0.00,0.0,74.0,94.0
2012-07-02,0.00,0.0,72.0,88.0
2012-07-03,0.00,0.0,71.0,89.0
2012-07-04,0.06,0.0,70.0,92.0
2012-07-05,0.00,0.0,77.0,95.0


### POIs

In [ ]:
poi_in  = "old_data/POIs.txt"
poi_out = "data/POIs_clean.csv"

# read whole POI file, large but manageable
pois = pd.read_csv(
    poi_in,
    sep="\t",
    header=None,
    names=["venue_id", "lat", "lon", "category", "country"],
    dtype={"venue_id": str, "category": str, "country": str},
)

# strip whitespace
for col in ["venue_id", "category", "country"]:
    pois[col] = pois[col].astype(str).str.strip()

# coerce lat/lon to numeric
pois["lat"] = pd.to_numeric(pois["lat"], errors="coerce")
pois["lon"] = pd.to_numeric(pois["lon"], errors="coerce")

# check on coordinates
pois = pois[
    (pois["lat"].between(-90, 90)) &
    (pois["lon"].between(-180, 180))
]

# drop rows missing venue_id or coords
pois = pois.dropna(subset=["venue_id", "lat", "lon"])

# keep minimal columns
pois_clean = pois[["venue_id", "lat", "lon"]].copy()

pois_clean.to_csv(poi_out, index=False)

pois_clean.head()


,venue_id,lat,lon
0,3fd66200f964a52000e71ee3,40.733596,-74.003139
1,3fd66200f964a52000e81ee3,40.758102,-73.975734
2,3fd66200f964a52000ea1ee3,40.732456,-74.003755
3,3fd66200f964a52000ec1ee3,42.345907,-71.087001
4,3fd66200f964a52000ee1ee3,39.933178,-75.159262


### Check-ins

In [ ]:
check_in = "old_data/Checkins.txt"
checkins_merged_out = "data/checkins_merged.csv"

chunk_size = 1000000
header_written = False

for chunk in pd.read_csv(
    check_in,
    sep="\t",
    header=None,
    names=["venue_id", "utc_time", "offset_min"],,
    chunksize=chunk_size,
):
    # strip whitespace
    for col in ["venue_id", "utc_time"]:
        chunk[col] = chunk[col].astype(str).str.strip()

    # clean offset and drop bad rows
    chunk["offset_min"] = pd.to_numeric(chunk["offset_min"], errors="coerce")
    chunk = chunk.dropna(subset=["venue_id", "utc_time", "offset_min"])

    # compute utc_dt/local_time/date on the chunk
    chunk["utc_dt"] = pd.to_datetime(chunk["utc_time"], errors="coerce")
    chunk = chunk.dropna(subset=["utc_dt"])
    chunk["local_time"] = chunk["utc_dt"] + pd.to_timedelta(chunk["offset_min"], unit="m")
    chunk["date"] = chunk["local_time"].dt.date

    # merge with POIs on the chunk
    chunk = chunk.merge(pois_clean[["venue_id", "lat", "lon"]],
                        on="venue_id", how="inner")

    # keep minimal cols
    chunk = chunk[["venue_id", "date", "lat", "lon"]]

    # append result to output CSV
    mode = "w" if not header_written else "a"
    chunk.to_csv(checkins_merged_out, mode=mode, index=False, header=not header_written)
    header_written = True

checkins_merged = pd.read_csv(checkins_merged_out)
checkins_merged.head()

/var/folders/6f/lxkzpyyn0fvcj0qtkc53g_2w0000gn/T/ipykernel_34226/1974790132.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["utc_dt"] = pd.to_datetime(chunk["utc_time"], errors="coerce")
/var/folders/6f/lxkzpyyn0fvcj0qtkc53g_2w0000gn/T/ipykernel_34226/1974790132.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["utc_dt"] = pd.to_datetime(chunk["utc_time"], errors="coerce")
/var/folders/6f/lxkzpyyn0fvcj0qtkc53g_2w0000gn/T/ipykernel_34226/1974790132.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["utc_dt"] = pd.to_datetime(chunk["utc_time"], err

,venue_id,date,lat,lon
0,4f5e3a72e4b053fd6a4313f6,2012-04-03,55.696132,37.557842
1,4b4b87b5f964a5204a9f26e3,2012-04-03,41.029717,28.974420
2,4a85b1b3f964a520eefe1fe3,2012-04-03,40.748939,-73.992280
3,4b4606f2f964a520751426e3,2012-04-03,30.270753,-97.752936
4,4c2b4e8a9a559c74832f0de2,2012-04-03,59.941041,30.308104


### Taxi

In [15]:
os.makedirs("data", exist_ok=True)

input_paths_2012 = [f"old_data/trip_data_{m}.csv" for m in range(7, 13)]

out_2012 = "data/taxi_clean_2012_H2.csv"

with open(out_2012, "w", encoding="utf8", newline="") as out:
    writer = csv.writer(out)
    writer.writerow(["pickup_datetime", "lon", "lat"])

    total_written = 0

    for input_path in input_paths_2012:
        with open(input_path, "r", encoding="utf8", errors="replace", newline="") as inp:
            reader = csv.reader(inp)
            header = next(reader, None)
            if not header:
                continue

            col = {name.strip().lower(): i for i, name in enumerate(header)}

            dt_i = col.get("pickup_datetime")
            lon_i = col.get("pickup_longitude")
            lat_i = col.get("pickup_latitude")

            if dt_i is None or lon_i is None or lat_i is None:
                raise ValueError(f"Missing expected columns in {input_path}. Header starts: {header[:20]}")

            for row in reader:
                if len(row) <= max(dt_i, lon_i, lat_i):
                    continue

                dt_raw = row[dt_i].strip()
                lon = row[lon_i].strip()
                lat = row[lat_i].strip()

                if not dt_raw or not lon or not lat:
                    continue

                # drop obvious bad coords
                if lon in ("0", "0.0") or lat in ("0", "0.0"):
                    continue

                writer.writerow([dt_raw, lon, lat])
                total_written += 1


In [19]:
import csv
import os

os.makedirs("data", exist_ok=True)

input_paths_2013 = [f"old_data/trip_data_{m}.csv" for m in range(1, 7)]
out_2013 = "data/taxi_clean_2013_H1.csv"

with open(out_2013, "w", encoding="utf8", newline="") as out:
    writer = csv.writer(out)
    writer.writerow(["pickup_datetime", "lon", "lat"])

    total_written = 0

    for input_path in input_paths_2013:
        print("Processing:", input_path)

        with open(input_path, "r", encoding="utf8", errors="replace", newline="") as inp:
            # strip NUL bytes while streaming
            reader = csv.reader((line.replace("\x00", "") for line in inp))

            header = next(reader, None)
            if not header:
                continue

            col = {name.strip().lower(): i for i, name in enumerate(header)}

            dt_i = col.get("pickup_datetime")
            lon_i = col.get("pickup_longitude")
            lat_i = col.get("pickup_latitude")

            if dt_i is None or lon_i is None or lat_i is None:
                raise ValueError(f"Missing expected columns in {input_path}. Header starts: {header[:20]}")

            for row in reader:
                if len(row) <= max(dt_i, lon_i, lat_i):
                    continue

                dt_raw = row[dt_i].strip()
                lon = row[lon_i].strip()
                lat = row[lat_i].strip()

                if not dt_raw or not lon or not lat:
                    continue

                if lon in ("0", "0.0") or lat in ("0", "0.0"):
                    continue

                writer.writerow([dt_raw, lon, lat])
                total_written += 1

print("Wrote:", out_2013, "rows:", total_written)


Processing: old_data/trip_data_1.csv
Processing: old_data/trip_data_2.csv
Processing: old_data/trip_data_3.csv
Processing: old_data/trip_data_4.csv
Processing: old_data/trip_data_5.csv
Processing: old_data/trip_data_6.csv
Wrote: data/taxi_clean_2013_H1.csv rows: 86260732


In [21]:
out_2012 = "data/taxi_clean_2012_H2.csv"
out_2013 = "data/taxi_clean_2013_H1.csv"
final_out = "data/taxi_clean.csv"

with open(final_out, "w", encoding="utf8", newline="") as out:
    writer = csv.writer(out)
    writer.writerow(["pickup_datetime", "lon", "lat"])

    for part in (out_2012, out_2013):
        with open(part, "r", encoding="utf8", errors="replace", newline="") as inp:
            reader = csv.reader(inp)
            next(reader, None)  # skip header
            for row in reader:
                if len(row) == 3:
                    writer.writerow(row)

taxi = pd.read_csv(final_out)
print("Final rows:", len(taxi))
taxi.head()


Final rows: 170629698


,pickup_datetime,lon,lat
0,2012-07-01 00:00:00,-73.872940,40.774117
1,2012-07-01 00:00:00,-73.983932,40.725529
2,2012-07-01 00:00:00,-73.994675,40.726028
3,2012-07-01 00:00:00,-73.979927,40.752216
4,2012-07-01 00:00:00,-73.979797,40.749989


## Integrating grid

- what if events overlap regions?

### Complaint

In [4]:
complaint = pd.read_csv('data/Complaint.csv')
grid_clipped = gpd.read_file('data/nyc_grid_2km.shp')

# Remove parentheses and split into two parts
lat_lon_split = complaint["Lat_Lon"].str.strip("()").str.split(",", expand=True)

complaint["lat"] = lat_lon_split[0].astype(float)
complaint["lon"] = lat_lon_split[1].astype(float)

gdf_complaint = gpd.GeoDataFrame(
    complaint,
    geometry=gpd.points_from_xy(complaint["lon"], complaint["lat"]),
    crs="EPSG:4326",       # WGS84
)

gdf_complaint = gdf_complaint.to_crs(grid_clipped.crs)  # use gdf_complaint

joined_complaint = gpd.sjoin(
    gdf_complaint,
    grid_clipped[["region_id", "geometry"]],
    how="inner",
    predicate="within",
)

joined_complaint["date"] = pd.to_datetime(joined_complaint["Date"]).dt.date 

complaint_daily = (
    joined_complaint
    .groupby(["region_id", "date"])
    .size()
    .rename("complaint_count")
    .reset_index()
)


### SAS

In [ ]:
sas_df = pd.read_csv('data/SAS.csv')
grid_clipped = gpd.read_file('data/nyc_grid_2km.shp')

# turn into GeoDataFrame, same CRS as grid
gdf_sas = gpd.GeoDataFrame(
    sas_df,
    geometry=gpd.points_from_xy(sas_df["xcoord"], sas_df["ycoord"]),
    crs=grid_clipped.crs
)

# spatial join: which grid cell is each stop in
sas_join = gpd.sjoin(
    gdf_sas,
    grid_clipped[["region_id", "geometry"]],
    how="inner",
    predicate="within",
)

sas_join["date"] = pd.to_datetime(sas_join["Date"]).dt.date 

# aggregate to region x day
sas_daily = (
    sas_join
    .groupby(["region_id", "date"])
    .size()
    .rename("sas_count")
    .reset_index()
)

### 311

In [ ]:
df_311 = pd.read_csv('data/311.csv')
grid_clipped = gpd.read_file('data/nyc_grid_2km.shp')

# Remove parentheses and split into two parts
lat_lon_split = df_311["Lat_Lon"].str.strip("()").str.split(",", expand=True)

df_311["lat"] = lat_lon_split[0].astype(float)
df_311["lon"] = lat_lon_split[1].astype(float)

gdf_311 = gpd.GeoDataFrame(
    df_311,
    geometry=gpd.points_from_xy(df_311["lon"], df_311["lat"]),
    crs="EPSG:4326",       # WGS84
)

gdf_311 = gdf_311.to_crs(grid_clipped.crs) # project to State Plane like grid

joined_311 = gpd.sjoin(
    gdf_311,
    grid_clipped[["region_id", "geometry"]],
    how="inner",
    predicate="within",
)

joined_311["date"] = pd.to_datetime(joined_311["Date"]).dt.date

daily_311 = (
    joined_311
    .groupby(["region_id", "date"])
    .size()
    .rename("311_count")
    .reset_index()
)

### Weather

In [ ]:
# Weather will be divided over all regions for every day
weather_clean = pd.read_csv('data/weather.csv')

### POIs

In [ ]:
# Static so no date join

pois_clean = pd.read_csv('data/POIs_clean.csv')
grid_clipped = gpd.read_file('data/nyc_grid_2km.shp')

gdf_pois = gpd.GeoDataFrame(
    pois_clean,
    geometry=gpd.points_from_xy(pois_clean["lon"], pois_clean["lat"]),
    crs="EPSG:4326",       # WGS84
)

gdf_pois = gdf_pois.to_crs(grid_clipped.crs) # project to State Plane like grid

joined_pois = gpd.sjoin(
    gdf_pois,
    grid_clipped[["region_id", "geometry"]],
    how="inner",
    predicate="within",
)

### Checkins

In [ ]:
checkins_merged = pd.read_csv('data/checkins_merged.csv')
grid_clipped = gpd.read_file('data/nyc_grid_2km.shp')

gdf_checkins = gpd.GeoDataFrame(
    checkins_merged,
    geometry=gpd.points_from_xy(checkins_merged["lon"], checkins_merged["lat"]),
    crs="EPSG:4326",       # WGS84
)

gdf_checkins = gdf_checkins.to_crs(grid_clipped.crs) # project to State Plane like grid

joined_checkins = gpd.sjoin(
    gdf_checkins,
    grid_clipped[["region_id", "geometry"]],
    how="inner",
    predicate="within",
)

joined_checkins["date"] = pd.to_datetime(joined_checkins["date"]).dt.date

checkin_daily = (
    joined_checkins
    .groupby(["region_id", "date"])
    .size()
    .rename("checkin_count")
    .reset_index()
)


### Taxi

In [2]:
import pandas as pd
import geopandas as gpd

# Read grid once, convert once to WGS84 so no reprojecting taxi points
grid_clipped = gpd.read_file("data/nyc_grid_2km.shp")[["region_id", "geometry"]].to_crs("EPSG:4326")

# Build spatial index once (speeds up sjoin)
_ = grid_clipped.sindex

chunksize = 1_000_000 

acc = None  # will hold running counts as a Series with MultiIndex (region_id, date)

for chunk in pd.read_csv(
    "data/taxi_clean.csv",
    usecols=["pickup_datetime", "lon", "lat"],
    dtype={"lon": "float32", "lat": "float32"},
    parse_dates=["pickup_datetime"],
    chunksize=chunksize,
):
    # drop bad/missing coords quickly
    chunk = chunk.dropna(subset=["pickup_datetime", "lon", "lat"])
    chunk = chunk[(chunk["lon"] != 0) & (chunk["lat"] != 0)]

    if chunk.empty:
        continue

    gdf = gpd.GeoDataFrame(
        chunk[["pickup_datetime"]].copy(),
        geometry=gpd.points_from_xy(chunk["lon"], chunk["lat"]),
        crs="EPSG:4326",
    )

    joined = gpd.sjoin(
        gdf,
        grid_clipped,
        how="inner",
        predicate="within",
    )

    # aggregate immediately to keep memory low
    joined["date"] = joined["pickup_datetime"].dt.date
    counts = joined.groupby(["region_id", "date"]).size()

    acc = counts if acc is None else acc.add(counts, fill_value=0)

# Final taxi_daily dataframe
taxi_daily = (
    acc.rename("taxi_count")
       .reset_index()
       .sort_values(["region_id", "date"])
       .reset_index(drop=True)
)

taxi_daily.head()


,region_id,date,taxi_count
0,0,2012-07-22,1.0
1,0,2012-08-01,1.0
2,0,2012-08-09,2.0
3,0,2012-08-16,1.0
4,0,2012-08-26,1.0


## Build feature matrix

In [ ]:
# Determine regions with crime
total_by_region = (
    complaint_daily
    .groupby("region_id")["complaint_count"]
    .sum()
)

active_regions = total_by_region[total_by_region > 0].index
len(active_regions) # dropped 284 - 250 = 44 regions


250

In [ ]:
# Filter all feature tables for active regions
grid_active        = grid_clipped[grid_clipped["region_id"].isin(active_regions)].copy()
complaint_daily    = complaint_daily[complaint_daily["region_id"].isin(active_regions)].copy()
sas_daily          = sas_daily[sas_daily["region_id"].isin(active_regions)].copy()
daily_311          = daily_311[daily_311["region_id"].isin(active_regions)].copy()
checkin_daily      = checkin_daily[checkin_daily["region_id"].isin(active_regions)].copy() 
taxi_daily         = taxi_daily[taxi_daily["region_id"].isin(active_regions)].copy()

In [69]:
# Build full index using only active regions
regions = sorted(active_regions)
dates = pd.date_range("2012-07-01", "2013-06-30", freq="D")

full_index = pd.MultiIndex.from_product(
    [regions, dates],
    names=["region_id", "date"]
)

X = pd.DataFrame(index=full_index).reset_index()


In [70]:
# Standardize 'date' to datetime
X['date'] = pd.to_datetime(X['date'])
sas_daily['date'] = pd.to_datetime(sas_daily['date'])
daily_311['date'] = pd.to_datetime(daily_311['date'])
checkin_daily['date'] = pd.to_datetime(checkin_daily['date'])
taxi_daily['date'] = pd.to_datetime(taxi_daily['date'])
weather_clean['DATE'] = pd.to_datetime(weather_clean['DATE'])

# merge features and fill missing counts with 0
X = X.merge(complaint_daily, on=["region_id", "date"], how="left")
X = X.merge(sas_daily, on=["region_id", "date"], how="left")
X = X.merge(daily_311, on=["region_id", "date"], how="left")
X = X.merge(checkin_daily, on=["region_id", "date"], how="left")
X = X.merge(taxi_daily, on=["region_id", "date"], how="left")
X = X.merge(weather_clean, left_on="date", right_on="DATE", how="left")

for col in ["complaint_count", "sas_count", "311_count", "checkin_count", "taxi_count"]:
    X[col] = X[col].fillna(0)


In [75]:
X.to_csv('data/FEATURE_MATRIX.csv')
grid_active.to_file("data/nyc_grid_2km_active.shp")

In [5]:
# UPDATE COMPLAINTS AND TAXI DATASETS

# Load existing feature matrix (defines region set and date window)
X = pd.read_csv("data/FEATURE_MATRIX.csv", index_col=0, parse_dates=["date"])

active_regions = set(X["region_id"].unique())
date_min = X["date"].min().normalize()
date_max = X["date"].max().normalize()

# Build exact already existing index, so updates cannot change the shape
full_idx = pd.MultiIndex.from_frame(X[["region_id", "date"]])

def align_to_X(daily_df, value_col):
    # Make types consistent
    daily_df = daily_df.copy()
    daily_df["region_id"] = daily_df["region_id"].astype(X["region_id"].dtype, errors="ignore")
    daily_df["date"] = pd.to_datetime(daily_df["date"]).dt.normalize()

    # Match notebook semantics: only active regions +
    #  within the X date range
    daily_df = daily_df[
        daily_df["region_id"].isin(active_regions)
        & (daily_df["date"] >= date_min)
        & (daily_df["date"] <= date_max)
    ]

    # Reindex to exactly X's (region_id, date) pairs, fill missing with 0
    s = (daily_df
         .set_index(["region_id", "date"])[value_col]
         .reindex(full_idx, fill_value=0))
    out = s.reset_index()
    out.columns = ["region_id", "date", value_col]
    return out

# complaint_daily must have columns: region_id, date, complaint_count
# taxi_daily must have columns: region_id, date, taxi_count
complaint_aligned = align_to_X(complaint_daily, "complaint_count")
taxi_aligned = align_to_X(taxi_daily, "taxi_count")

# Merge in the updated columns only
X = X.drop(columns=["complaint_count", "taxi_count"], errors="ignore")
X = X.merge(complaint_aligned, on=["region_id", "date"], how="left")
X = X.merge(taxi_aligned, on=["region_id", "date"], how="left")

X.to_csv("data/FEATURE_MATRIX.csv")

